from langchain_core.messages import HumanMessage## 多模态的定义
多模态（Multimodality）指 LLM 处理一份来自于多形式的数据（例如文本+音频、视频+图片或是各自的组合），需要 LLM 能获取无缝地处理这些数据。

多模态任务主要有三个：
* Chat Models：指 LLM 可以接受多模态的输入和输出，能够解决不同类型的数据之间的关系；
* Embedding Models：指 LLM 可以嵌入这些多模态的数据形成一些特定的向量；
* Vector Stores：将不同模态的数据进行嵌入，并且能够存储，成为后续 Retrieve 的关键。

具体可以参考[官方文档](https://python.langchain.com/docs/concepts/multimodality/#overview)，同时是否支持多模态可以参考这张[表格](https://python.langchain.com/docs/integrations/chat/#featured-providers)。

In [2]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI

# 我们仍旧使用 GPT-4o，这是一个支持多模态的 LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,  # 让我们的 LLM 最准确
    max_tokens=2048,
    timeout=None,
    max_retries=2,
)

我们做一个简单的任务，将一张图片传递给 LLM，让它帮我们介绍一下图片的内容，我们使用这张图片，通过 URL 传入到模型中去。

<img src="https://images.squarespace-cdn.com/content/v1/5c81f8d10b77bd7cfa2c6904/1604083180558-G7Q78M6D1LDLTDAKOR9D/IMG_2445.JPG" alt="UCSD Fallen Star" style="width: 60%">

In [3]:
img_url = "https://images.squarespace-cdn.com/content/v1/5c81f8d10b77bd7cfa2c6904/1604083180558-G7Q78M6D1LDLTDAKOR9D/IMG_2445.JPG"

In [7]:
import base64
import httpx


# 这是一套标准的下载图片，然后把图片转成 Base64 编码的字符串流程
def get_b64_img_data(url: str) -> str:
    # 向指定图片链接发 GET 请求，并获取响应的二进制内容（也就是图片原始字节）
    # 再把图片的二进制数据转成 base64 编码（bytes 类型）
    return base64.b64encode(httpx.get(url).content).decode("utf-8")


image_data = get_b64_img_data(img_url)

In [9]:
from langchain_core.messages import HumanMessage

message = [
    HumanMessage(
        content=[{
            "type": "text",
            "text": "describe the house which is interesting in this image."
        },
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}
            }]
    )
]
response = llm.invoke(message)
print(response.content)

The image shows a unique architectural feature where a small, traditional-looking house is perched precariously on the edge of a modern, concrete building. The house appears to be a typical residential structure with a pitched roof and windows, contrasting sharply with the angular, industrial design of the building it sits on. This juxtaposition creates an intriguing visual effect, making the house appear as if it is teetering on the brink of falling off. The sky is clear and blue, adding to the striking appearance of the scene.


也可以直接使用这样的方式，即直接输入 URL，但是只有部分 Provider 能提供这样的功能。
```python
HumanMessage(
    content=[
        {"type": "text", "text": "describe the house in this image"},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
```
具体如何传入参数，可以参考OpenAI的官方文档[Images and vision](https://platform.openai.com/docs/guides/images?api-mode=responses&format=url).

我们来分析一下刚刚运行的代码，它的大体架构是这样的。content 属性是一个 List，可以传入若干条信息，可以是文本，也可以是各种各样的文本。然后 HumanMessage 就可以整合一整个输入。
```python
message = HumanMessage(
    content=[
        {"type": "text", "text": "...text1..."},
        {"type": "image_url", "image_url": "...url1..."},
        {"type": "text", "text": "...text2..."},
        {"type": "image_url", "image_url": "...url2..."},
        {"type": "text", "text": "...text3..."},
        ...
    ]
)
```
我们再加一张图片，这是刚刚那个小房子的另一个视角，同时我们这次传入 URL，不用 base64 编码。我们传入三段文本，两张图片，测试 LLM 的多模态能力。

<img src="https://i.pinimg.com/736x/cd/1f/4e/cd1f4efca99d5cb95065c5877c263515.jpg" alt="Another view of Fallen Star" style="width: 60%">

In [11]:
img_url_2 = "https://i.pinimg.com/736x/cd/1f/4e/cd1f4efca99d5cb95065c5877c263515.jpg"
message = [
    HumanMessage(
        content=[
            {"type": "text", "text": "This is the first photo."},
            {"type": "image_url", "image_url": {"url": img_url}},
            {"type": "text", "text": "This is the second photo."},
            {"type": "image_url", "image_url": {"url": img_url_2}},
            {"type": "text", "text": "Are the small houses in these photos the same.?"},
        ]
    )
]
response = llm.invoke(message)
print(response.content)

Yes, the small houses in both photos appear to be the same. They are both perched on the edge of a building, which is a distinctive architectural feature.


与此同时，多模态的 LLM 可以和 tool 相结合。比如这里我使用了 `Literal[...]` 来约束 LLM 的返回值。在底层 LangChain 会将这个参数转化为 `function schema` 中的枚举值，从而 LLM 会为我的输入做一个最合适的枚举项（天气），作为 tool_call 的返回值。

In [20]:
from typing import Literal

from langchain_core.tools import tool


@tool
def weather_tool(weather: Literal["sunny", "cloudy", "rainy", "unknown"]):
    """Describe the weather in this picture."""
    return weather


model_with_tools = llm.bind_tools([weather_tool])

message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {"type": "image_url", "image_url": {"url": img_url}},
    ],
)
response = model_with_tools.invoke([message])
print(response.tool_calls)
print(response.tool_calls[0]['args'])

[{'name': 'weather_tool', 'args': {'weather': 'sunny'}, 'id': 'call_yN6Gc6mTeHu7kDdZ8V1BbVSH', 'type': 'tool_call'}]
{'weather': 'sunny'}


为了不让这部分笔记早早结束，我们试试高级功能，用一用 `|` 符号。这个符号的意思是调链式，`prompt | llm` 表示一个链式组合：把 prompt 生成的提示作为输入，传给 llm 进行调用。就像“流水线拼接”一样构建完整的调用链。

In [50]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage

# 我们改变一下 LLM 的 temp，让它变得更加有创造性一些
llm_with_higher_temp = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7
)

prompt = ChatPromptTemplate([
    SystemMessage(
        content="You are a helpful assistant. Your name is Bob and your tone should be the same as the 'Bob' in the game 'Hearthstone'. You should be talkative and humorous, sometimes you can mention some words in the game 'Battlegrounds' to entertain the conversation. And you should reply in Chinese (simplified)."),
    HumanMessage(
        content=[
            {"type": "text", "text": "What can you see in the photo?"},
            {"type": "image_url", "image_url": {"url": img_url}},
        ]
    )]
)

In [51]:
# 把 prompt 的输出作为输入传递给 model，组合成一个新的 chain
chain = prompt | llm_with_higher_temp
response = chain.invoke({}) # <-- 因为我们没有使用格式化变量
print(response.content)

这张照片中是一座现代风格的建筑，建筑顶部有一间看似悬挂的小房子。天空是晴朗的蓝色，建筑周围有一些树木。这种设计看起来非常独特和有创意，就像在《炉石传说：酒馆战棋》里玩出了意想不到的组合一样！


🤔思考题：
我可以这样使用 template 的格式化变量吗？
```python
prompt = ChatPromptTemplate([
    SystemMessage(
        content="..."),
    HumanMessage(
        content=[
            {"type": "text", "text": "What can you see in the photo?"},
            {"type": "image_url", "image_url": {"url": {input_img_url}}},
        ]
    )]
)
# 在 invoke 的时候通过 input_img_url 输入我们的变量 img_url
response = chain.invoke({"input_img_url" : img_url})
```
答案是不行，虽然这样做符合 Template 定义，但是 LangChain 的 `ChatPromptTemplate` 模板变量只能替换在 `{...}` 里，而不能出现在深层嵌套结构中。因此这样写会报错.
